# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [7]:
# Remover colunas irrelevantes
df = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])

# Adicionar uma coluna com o logaritmo da renda
df['log_renda'] = np.log(df['renda'])

# Criar a fórmula para incluir todas as variáveis, com patsy convertendo qualitativas em dummies
# Isso mantém automaticamente a categoria mais frequente como referência
formula = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

# Criar as matrizes para o modelo
y, X = patsy.dmatrices(formula, df, return_type='dataframe')

# Ajustar o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Exibir o resumo do modelo
print(modelo.summary())


                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 01 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:29:23   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [9]:
# Atualizar a fórmula removendo a variável educacao[T.Secundário]
formula_refinada = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + ' \
                   'educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia - educacao[T.Secundário]'

# Criar as novas matrizes para o modelo refinado
y_refinado, X_refinado = patsy.dmatrices(formula_refinada, df, return_type='dataframe')

# Ajustar o modelo refinado
modelo_refinado = sm.OLS(y_refinado, X_refinado).fit()

# Exibir o resumo do modelo refinado
print(modelo_refinado.summary())


                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 01 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:31:06   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [11]:
# Removendo variáveis iterativamente e ajustando o modelo
variaveis_a_remover = [
    'educacao[T.Pós graduação]',
    'tipo_residencia[T.Com os pais]',
    'tipo_residencia[T.Comunitário]',
    'tipo_residencia[T.Estúdio]',
    'tipo_residencia[T.Governamental]',
    'tipo_renda[T.Bolsista]',
    'tipo_renda[T.Pensionista]'
]

# Ajuste do modelo após cada remoção
formula = formula_refinada  # Iniciando com a última fórmula refinada
for variavel in variaveis_a_remover:
    # Remover a variável atual
    formula = formula.replace(f' + {variavel}', '')

    # Criar novas matrizes para o modelo atualizado
    y_temp, X_temp = patsy.dmatrices(formula, df, return_type='dataframe')
    
    # Ajustar o modelo com a variável removida
    modelo_temp = sm.OLS(y_temp, X_temp).fit()
    
    # Exibir o resumo do modelo atualizado
    print(f'Resumo do modelo após remover {variavel}:')
    print(modelo_temp.summary())
    print("\n" + "="*80 + "\n")


Resumo do modelo após remover educacao[T.Pós graduação]:
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 01 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:33:45   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

Modelo	R-quadrado ajustado	AIC	BIC
Modelo 1	0.356	27190	27367
Modelo 2	0.356	27188	27361
Modelo 3	0.356	27188	27361
Modelo 4	0.356	27188	27361
Modelo 5	0.356	27186	27359

 A análise dos critérios de informação (AIC e BIC) indica que o Modelo 5 apresenta a melhor combinação entre ajuste aos dados e complexidade. Embora o poder explicativo geral dos modelos seja similar, conforme evidenciado pelo R² ajustado, o Modelo 5, ao eliminar as variáveis "tipo_residencia" e "educacao", demonstra ser mais simplificado sem comprometer significativamente sua capacidade de explicar a variabilidade da variável dependente.